In [29]:
# cell 1
import ee
ee.Authenticate()  # ยืนยันตัวตนกับ Google Earth Engine
project_id = "ee-sakda-451407"

ee.Initialize(project=project_id)  # เริ่มต้นใช้งานด้วย project ID ที่กำหนด


In [30]:
# cell 2
!pip install folium

In [31]:
# cell 3 folium
import folium   

def add_raster_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

# เพิ่มฟังก์ชัน add_raster_layer เข้าไปใน folium.Map
folium.Map.add_raster_layer = add_raster_layer

def add_vector_layer(self, ee_feature_collection, vis_params, name):
    map_id_dict = ee.FeatureCollection(ee_feature_collection).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True
    ).add_to(self)


# เพิ่มฟังก์ชัน add_vector_layer เข้าไปใน folium.Map
folium.Map.add_vector_layer = add_vector_layer  


In [32]:
# cell 4 geometry, feature, feature_collection

# สร้างจุด (Point) ที่พิกัดที่กำหนด
point = ee.Geometry.Point(98.9853, 18.7883)
# สร้างฟีเจอร์ (Feature) จากจุดที่สร้างขึ้น
feature = ee.Feature(point, {'name': 'My Point'})
# สร้างคอลเลกชันของฟีเจอร์ (FeatureCollection)
feature_collection = ee.FeatureCollection([feature])

print('Point:', point.getInfo())
print('Feature:', feature.getInfo())
print('FeatureCollection:', feature_collection.getInfo())


Point: {'type': 'Point', 'coordinates': [98.9853, 18.7883]}
Feature: {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [98.9853, 18.7883]}, 'properties': {'name': 'My Point'}}
FeatureCollection: {'type': 'FeatureCollection', 'columns': {'name': 'String', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [98.9853, 18.7883]}, 'id': '0', 'properties': {'name': 'My Point'}}]}


In [33]:
# cell 5
# show vector data on map
fig1 = folium.Figure(height="300px")
m2 = folium.Map(location=[18.7883, 98.9853], zoom_start=14).add_to(fig1)

# กำหนดพารามิเตอร์การแสดงผลสำหรับฟีเจอร์คอลเลกชัน
vector_vis_params = {
    'color': 'red',
    'pointRadius': 10,
    'width': 2
}
# เพิ่มฟีเจอร์คอลเลกชันลงในแผนที่ด้วยพารามิเตอร์การแสดงผลที่กำหนด
m2.add_vector_layer(feature_collection, vector_vis_params, 'My Feature Collection')

folium.LayerControl().add_to(m2)

# แสดงแผนที่ผ่าน Figure เพื่อให้ render เป็น HTML
fig1


In [34]:
# cell 6 image
bbox = ee.Geometry.Rectangle([98.9, 18.7, 99.1, 18.9])
# ดึงข้อมูลภาพจาก Sentinel-2 วันที่ 2024-01-09 ที่ครอบคลุมพื้นที่ที่กำหนด
s2_image = ee.Image('COPERNICUS/S2/20240109T035129_20240109T040305_T47QMA') \
    .clip(bbox)  # ตัดภาพให้ตรงกับพื้นที่ที่กำหนด

print('Bands:', s2_image.bandNames().getInfo())
print('วันที่:', s2_image.date().format('YYYY-MM-dd').getInfo())

# กำหนดพารามิเตอร์การแสดงผลสำหรับภาพ Sentinel-2
s2_vis_params = {
    'bands': ['B4', 'B3', 'B2'],  # แสดงผลด้วยแถบสีแดง, เขียว, น้ำเงิน
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}
# สร้างแผนที่ใหม่สำหรับแสดงภาพ Sentinel-2
fig2 = folium.Figure(height="300px")
m3 = folium.Map(location=[18.8, 98.95], zoom_start=12).add_to(fig2)
# เพิ่มภาพ Sentinel-2 ลงในแผนที่ด้วยพารามิเตอร์การแสดงผลที่กำหนด
m3.add_raster_layer(s2_image, s2_vis_params, 'Sentinel-2 Image')
folium.LayerControl().add_to(m3)    
# แสดงแผนที่ผ่าน Figure เพื่อให้ render เป็น HTML
fig2



Bands: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60']
วันที่: 2024-01-09


In [35]:
# cell 7 - Image
# โหลดภาพ Sentinel-2 และแสดงผลบนแผนที่

# กำหนดพื้นที่สนใจ (บริเวณเชียงใหม่)
bbox = ee.Geometry.Rectangle([98.9, 18.7, 99.1, 18.9])

# โหลด ImageCollection ของ Sentinel-2 และกรองตามวันที่และพื้นที่
s2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2024-01-01', '2024-12-31') \
    .filterBounds(bbox) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# เลือกภาพที่มีเมฆน้อยที่สุด
img = s2.sort('CLOUDY_PIXEL_PERCENTAGE').median()
print('ภาพที่เลือก:', img.getInfo())
print('จำนวนภาพ:', s2.size().getInfo())
print('Bands:', img.bandNames().getInfo())
print('วันที่เริ่มต้น:', s2.first().date().format('YYYY-MM-dd').getInfo())
print('วันที่สิ้นสุด:', s2.sort('CLOUDY_PIXEL_PERCENTAGE').first().date().format('YYYY-MM-dd').getInfo())

# แสดงภาพบนแผนที่
fig3 = folium.Figure(height="300px")
m3 = folium.Map(location=[18.7883, 98.9853], zoom_start=12).add_to(fig3)

# แสดง True Color (B4, B3, B2)
m3.add_raster_layer(img, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'True Color')

# แสดง False Color (B8, B4, B3) - เน้นพืชพรรณ
m3.add_raster_layer(img, {'bands': ['B8', 'B4', 'B3'], 'min': 0, 'max': 3000}, 'False Color')

folium.LayerControl().add_to(m3)

fig3


ภาพที่เลือก: {'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B7', 'data_type': {'type': 

In [36]:
# cell 8 - คำนวณ NDVI และแสดงผล
fig4 = folium.Figure(height="300px")
m4 = folium.Map(location=[18.7883, 98.9853], zoom_start=12).add_to(fig4)

ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')
ndvi_palette = ['red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen']
m4.add_raster_layer(ndvi, {'min': -0.2, 'max': 0.8, 'palette': ndvi_palette}, 'NDVI')

folium.LayerControl().add_to(m4)
fig4